In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import tree
# import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML
from sklearn.metrics import accuracy_score, classification_report


In [2]:
df = pd.read_csv("./data/modified/weather_arrivals_departures_merged_correct_tz.csv")
# df['arrival_status'] = pd.cut(df['Arrival Delay (Minutes)'], bins=[float('-inf'), -5, 5.00001, float('inf')], labels=["early", "ontime", "late"], right=False)
df

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Taxi-In time (Minutes),Delay Carrier (Minutes)_x,Delay Weather (Minutes)_x,Delay National Aviation System (Minutes)_x,Delay Security (Minutes)_x,Delay Late Aircraft Arrival (Minutes)_x,scheduled_arrival_time,actual_arrival_time,date_x,date_time_x,app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,timestamp_local_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,key_x,date_time_map_x,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Taxi-Out time (Minutes),Delay Carrier (Minutes)_y,Delay Weather (Minutes)_y,Delay National Aviation System (Minutes)_y,Delay Security (Minutes)_y,Delay Late Aircraft Arrival (Minutes)_y,scheduled_departure_time,actual_departure_time,date_y,date_time_y,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,timestamp_local_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,key_y,date_time_map_y,arrival_status
0,B6,01/01/2020,116,JFK,08:22,08:08,75,62,-14,3,0,0,0,0,0,2020-01-01 08:22:00,2020-01-01 08:08:00,2020-01-01,2020-01-01 08:22:00,-6.50,100,-6.20,989,65,-0.40,2020-01-01 08:15:00,0.40,16,804,250,12.10,7.07,SYR,2020-01-01 08:15:00,SYR,07:07,07:06,75,62,-1,11,0,0,0,0,0,2020-01-01 07:07:00,2020-01-01 07:06:00,2020-01-01,2020-01-01 07:07:00,-2.60,87,-2.30,1005,69,2.80,2020-01-01 07:00:00,0.00,16,804,280,11.10,7.70,JFK,2020-01-01 07:00:00,early
1,OO,01/01/2020,3247,ORD,10:43,10:40,113,110,-3,4,0,0,0,0,0,2020-01-01 10:43:00,2020-01-01 10:40:00,2020-01-01,2020-01-01 10:43:00,-5.00,100,-5.20,990,65,0.60,2020-01-01 10:45:00,0.60,16,804,255,14.90,6.70,SYR,2020-01-01 10:45:00,SYR,07:50,07:50,113,110,0,28,0,0,0,0,0,2020-01-01 08:50:00,2020-01-01 08:50:00,2020-01-01,2020-01-01 07:50:00,-12.20,25,-7.90,984,87,-6.10,2020-01-01 07:45:00,1.00,16,802,200,6.60,4.59,ORD,2020-01-01 07:45:00,ontime
2,9E,01/01/2020,5066,JFK,11:37,11:03,92,72,-34,5,0,0,0,0,0,2020-01-01 11:37:00,2020-01-01 11:03:00,2020-01-01,2020-01-01 11:37:00,-5.10,100,-5.40,990,63,0.80,2020-01-01 11:30:00,0.60,16,804,265,12.60,7.75,SYR,2020-01-01 11:30:00,SYR,10:05,09:51,92,72,-14,20,0,0,0,0,0,2020-01-01 10:05:00,2020-01-01 09:51:00,2020-01-01,2020-01-01 10:05:00,-1.20,87,-5.80,1007,49,3.90,2020-01-01 10:00:00,0.70,16,804,270,11.60,7.70,JFK,2020-01-01 10:00:00,early
3,F9,01/01/2020,1214,MCO,14:39,14:23,164,156,-16,6,0,0,0,0,0,2020-01-01 14:39:00,2020-01-01 14:23:00,2020-01-01,2020-01-01 14:39:00,-4.40,100,-5.60,991,60,1.30,2020-01-01 14:45:00,0.50,16,804,260,12.00,7.32,SYR,2020-01-01 14:45:00,SYR,11:55,11:47,164,156,-8,14,0,0,0,0,0,2020-01-01 11:55:00,2020-01-01 11:47:00,2020-01-01,2020-01-01 11:55:00,18.30,25,2.60,1016,35,18.30,2020-01-01 12:00:00,4.00,16,802,290,5.20,3.10,MCO,2020-01-01 12:00:00,early
4,OO,01/01/2020,3253,ORD,14:43,14:38,113,110,-5,5,0,0,0,0,0,2020-01-01 14:43:00,2020-01-01 14:38:00,2020-01-01,2020-01-01 14:43:00,-4.40,100,-5.60,991,60,1.30,2020-01-01 14:45:00,0.50,16,804,260,12.00,7.32,SYR,2020-01-01 14:45:00,SYR,11:50,11:48,113,110,-2,25,0,0,0,0,0,2020-01-01 12:50:00,2020-01-01 12:48:00,2020-01-01,2020-01-01 11:50:00,-2.80,25,-5.70,982,58,1.70,2020-01-01 11:45:00,1.80,16,802,200,10.80,5.09,ORD,2020-01-01 11:45:00,ontime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,YX,12/31/2023,3719,ORD,16:44,17:18,124,157,34,7,0,0,34,0,0,2023-12-31 16:44:00,2023-12-31 17:18:00,2023-12-31,2023-12-31 16:44:00,-5.30,100,-3.40,992,88,-1.70,2023-12-31 16:45:00,0.00,14,804,270,2.72,2.72,SYR,2023-12-31 16:45:00,SYR,13:40,13:41,124,157,1,58,0,0,34,0,0,2023-12-31 14:40:00,2023-12-31 14:41:00,2023-12-31,2023-12-31 13:40:0

In [3]:
def convert_to_timestamp(df):
    if "scheduled_departure_time" in df.columns:
        df["scheduled_departure_time"] = pd.to_datetime(df["scheduled_departure_time"])
        
    if "scheduled_arrival_time" in df.columns:
        df["scheduled_arrival_time"] = pd.to_datetime(df["scheduled_arrival_time"])
    
    if "actual_departure_time" in df.columns:
        df["actual_departure_time"] = pd.to_datetime(df["actual_departure_time"])
        
    if "actual_arrival_time" in df.columns:
        df["actual_arrival_time"] = pd.to_datetime(df["actual_arrival_time"])
    
    return df


In [4]:
# Adding 1hr to ORD to convert to EST from CST
# df.loc[df["Origin Airport"]=="ORD", "scheduled_departure_time"] += pd.Timedelta(hours=1)
# df.loc[df["Origin Airport"]=="ORD", "actual_departure_time"] += pd.Timedelta(hours=1)


In [5]:
# df["diff1"] = (df["scheduled_arrival_time"]-df["scheduled_departure_time"]).dt.total_seconds() // 60
# df["diff2"] = (df["actual_arrival_time"]-df["actual_departure_time"]).dt.total_seconds() // 60


# df[df["Scheduled Elapsed Time (Minutes)"] != df["diff"]][["Scheduled Arrival Time", "scheduled_arrival_time", "Scheduled departure time", "scheduled_departure_time", "Scheduled elapsed time (Minutes)", "Scheduled Elapsed Time (Minutes)", "diff"]]
# df[df["Scheduled Elapsed Time (Minutes)"] != df["diff1"]][["Scheduled Arrival Time", "scheduled_arrival_time", "Scheduled departure time", "scheduled_departure_time", "Scheduled elapsed time (Minutes)", "Scheduled Elapsed Time (Minutes)", "diff1"]]
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff2"]) & (df["Actual Elapsed Time (Minutes)"] != 0)][["Actual Arrival Time", "Actual departure time", 'Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', "Actual elapsed time (Minutes)", "Actual Elapsed Time (Minutes)", "diff2", "actual_departure_time", "actual_arrival_time"]]
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] != 0)][["Actual Arrival Time", "Actual departure time", 'Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', "Actual elapsed time (Minutes)", "Actual Elapsed Time (Minutes)", "diff", "actual_departure_time", "actual_arrival_time"]]
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] != 0) & (df["Actual Elapsed Time (Minutes)"] == df["diff2"])][["Actual Arrival Time", "Actual departure time", 'Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', "Actual elapsed time (Minutes)", "Actual Elapsed Time (Minutes)", "diff", "diff2", "actual_departure_time", "actual_arrival_time"]]
# df.loc[df["Scheduled Elapsed Time (Minutes)"] != df["diff1"], "scheduled_arrival_time"] += pd.Timedelta(days=1)
# df.loc[(df["Actual Elapsed Time (Minutes)"] != df["diff2"]) & (df["Actual Elapsed Time (Minutes)"] != 0), "actual_arrival_time"] += pd.Timedelta(days=1)

# df.drop(columns="diff1", inplace=True)
# df.drop(columns="diff2", inplace=True)
# df["diff2"] = df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] != 0)]["diff"] + 1440



In [6]:
 # df[(df["Actual Elapsed Time (Minutes)"]==0)]

In [7]:
# pd.concat([incor, df[df["Actual elapsed time (Minutes)"]==0]]).reset_index(drop=True).to_csv("./data/modified/incorrect_flight_data.csv", index=False)

In [8]:
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] == 0)].to_csv("./data/modified/incorrect_flight_data.csv")
# df = df[~((df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] == 0))]
# df = df[~(df["Actual Elapsed Time (Minutes)"]==0)].reset_index(drop=True)
# df
# df.drop(columns="diff", inplace=True)
# df.to_csv("./data/modified/weather_arrivals_departures_merged_correct_tz.csv", index=False)

In [9]:
df = convert_to_timestamp(df)

In [10]:
numeric_cols = df.select_dtypes(include='number').columns
numeric_cols

Index(['Flight Number', 'Scheduled Elapsed Time (Minutes)',
       'Actual Elapsed Time (Minutes)', 'Arrival Delay (Minutes)',
       'Taxi-In time (Minutes)', 'Delay Carrier (Minutes)_x',
       'Delay Weather (Minutes)_x',
       'Delay National Aviation System (Minutes)_x',
       'Delay Security (Minutes)_x', 'Delay Late Aircraft Arrival (Minutes)_x',
       'app_temp_x', 'clouds_x', 'dewpt_x', 'pres_x', 'rh_x', 'temp_x', 'uv_x',
       'vis_x', 'weather.code_x', 'wind_dir_x', 'wind_gust_spd_x',
       'wind_spd_x', 'Scheduled elapsed time (Minutes)',
       'Actual elapsed time (Minutes)', 'Departure delay (Minutes)',
       'Taxi-Out time (Minutes)', 'Delay Carrier (Minutes)_y',
       'Delay Weather (Minutes)_y',
       'Delay National Aviation System (Minutes)_y',
       'Delay Security (Minutes)_y', 'Delay Late Aircraft Arrival (Minutes)_y',
       'app_temp_y', 'clouds_y', 'dewpt_y', 'pres_y', 'rh_y', 'temp_y', 'uv_y',
       'vis_y', 'weather.code_y', 'wind_dir_y', 'wind_gus

In [11]:
cat_cols = df.select_dtypes(exclude='number').columns
cat_cols

Index(['Carrier Code', 'Date (MM/DD/YYYY)', 'Origin Airport',
       'Scheduled Arrival Time', 'Actual Arrival Time',
       'scheduled_arrival_time', 'actual_arrival_time', 'date_x',
       'date_time_x', 'timestamp_local_x', 'key_x', 'date_time_map_x',
       'Destination Airport', 'Scheduled departure time',
       'Actual departure time', 'scheduled_departure_time',
       'actual_departure_time', 'date_y', 'date_time_y', 'timestamp_local_y',
       'key_y', 'date_time_map_y', 'arrival_status'],
      dtype='object')

In [12]:
cat_drop_cols = ["Date (MM/DD/YYYY)", "Scheduled Arrival Time", "Actual Arrival Time", "date_x", 
             "date_time_x", "timestamp_local_x", "key_x", "date_time_map_x", "Destination Airport", 
             "Scheduled departure time", "Actual departure time",'date_y', 'date_time_y', 'timestamp_local_y',
             'key_y', 'date_time_map_y' 
            ]

numeric_drop_cols = ['Flight Number', 'Actual Elapsed Time (Minutes)',
                     'Taxi-In time (Minutes)', 'Delay Carrier (Minutes)_x','Delay Weather (Minutes)_x', 
                     'Delay National Aviation System (Minutes)_x', 'Delay Security (Minutes)_x', 'Delay Late Aircraft Arrival (Minutes)_x',
                     'Scheduled elapsed time (Minutes)','Actual elapsed time (Minutes)', 'Departure delay (Minutes)','Taxi-Out time (Minutes)', 
                     'Delay Carrier (Minutes)_y','Delay Weather (Minutes)_y','Delay National Aviation System (Minutes)_y',
                     'Delay Security (Minutes)_y', 'Delay Late Aircraft Arrival (Minutes)_y']

In [35]:
model_df = df.drop(columns=cat_drop_cols + numeric_drop_cols + ["actual_arrival_time", "actual_departure_time"]).copy()
model_df

,Carrier Code,Origin Airport,Scheduled Elapsed Time (Minutes),Arrival Delay (Minutes),scheduled_arrival_time,app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,scheduled_departure_time,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,arrival_status
0,B6,JFK,75,-14,2020-01-01 08:22:00,-6.50,100,-6.20,989,65,-0.40,0.40,16,804,250,12.10,7.07,2020-01-01 07:07:00,-2.60,87,-2.30,1005,69,2.80,0.00,16,804,280,11.10,7.70,early
1,OO,ORD,113,-3,2020-01-01 10:43:00,-5.00,100,-5.20,990,65,0.60,0.60,16,804,255,14.90,6.70,2020-01-01 08:50:00,-12.20,25,-7.90,984,87,-6.10,1.00,16,802,200,6.60,4.59,ontime
2,9E,JFK,92,-34,2020-01-01 11:37:00,-5.10,100,-5.40,990,63,0.80,0.60,16,804,265,12.60,7.75,2020-01-01 10:05:00,-1.20,87,-5.80,1007,49,3.90,0.70,16,804,270,11.60,7.70,early
3,F9,MCO,164,-16,2020-01-01 14:39:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 11:55:00,18.30,25,2.60,1016,35,18.30,4.00,16,802,290,5.20,3.10,early
4,OO,ORD,113,-5,2020-01-01 14:43:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 12:50:00,-2.80,25,-5.70,982,58,1.70,1.80,16,802,200,10.80,5.09,ontime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,YX,ORD,124,34,2023-12-31 16:44:00,-5.30,100,-3.40,992,88,-1.70,0.00,14,804,270,2.72,2.72,2023-12-31 14:40:00,-2.40,100,-2.10,989,80,1.00,0.50,4,804,310,7.40,3.10,late
11776,MQ,ORD,106,102,2023-12-31 17:28:00,-4.50,100,-3.70,991,86,-1.70,0.00,15,804,355,2.04,2.04,2023-12-31 15:42:00,-2.30,100,-2.30,989,78,1.10,0.40,10,804,300,6.60,3.10,late
11777,B6,MCO,175,-36,2023-12-31 19:27:00,-3.40,100,-4.00,1000,86,-1.90,0.00,15,804,360,2.40,1.20,2023-12-31 16:32:00,18.10,80,8.80,1015,52,18.90,1.00,16,804,305,4.40,4.15,early
11778,UA,ORD,114,-10,2023-12-31 21:09:00,-4.30,100,-3.90,1000,88,-2.20,0.00,16,804,360,3.30,1.50,2023-12-31 19:15:00,-5.40,87,-2.30,993,82,0.40,0.00,13,804,315,10.70,7.03,early


In [36]:
numeric_mod_cols = model_df.select_dtypes(include="number").columns
cat_mod_cols = model_df.select_dtypes(exclude="number").columns

In [37]:
model_df

,Carrier Code,Origin Airport,Scheduled Elapsed Time (Minutes),Arrival Delay (Minutes),scheduled_arrival_time,app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,scheduled_departure_time,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,arrival_status
0,B6,JFK,75,-14,2020-01-01 08:22:00,-6.50,100,-6.20,989,65,-0.40,0.40,16,804,250,12.10,7.07,2020-01-01 07:07:00,-2.60,87,-2.30,1005,69,2.80,0.00,16,804,280,11.10,7.70,early
1,OO,ORD,113,-3,2020-01-01 10:43:00,-5.00,100,-5.20,990,65,0.60,0.60,16,804,255,14.90,6.70,2020-01-01 08:50:00,-12.20,25,-7.90,984,87,-6.10,1.00,16,802,200,6.60,4.59,ontime
2,9E,JFK,92,-34,2020-01-01 11:37:00,-5.10,100,-5.40,990,63,0.80,0.60,16,804,265,12.60,7.75,2020-01-01 10:05:00,-1.20,87,-5.80,1007,49,3.90,0.70,16,804,270,11.60,7.70,early
3,F9,MCO,164,-16,2020-01-01 14:39:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 11:55:00,18.30,25,2.60,1016,35,18.30,4.00,16,802,290,5.20,3.10,early
4,OO,ORD,113,-5,2020-01-01 14:43:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 12:50:00,-2.80,25,-5.70,982,58,1.70,1.80,16,802,200,10.80,5.09,ontime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,YX,ORD,124,34,2023-12-31 16:44:00,-5.30,100,-3.40,992,88,-1.70,0.00,14,804,270,2.72,2.72,2023-12-31 14:40:00,-2.40,100,-2.10,989,80,1.00,0.50,4,804,310,7.40,3.10,late
11776,MQ,ORD,106,102,2023-12-31 17:28:00,-4.50,100,-3.70,991,86,-1.70,0.00,15,804,355,2.04,2.04,2023-12-31 15:42:00,-2.30,100,-2.30,989,78,1.10,0.40,10,804,300,6.60,3.10,late
11777,B6,MCO,175,-36,2023-12-31 19:27:00,-3.40,100,-4.00,1000,86,-1.90,0.00,15,804,360,2.40,1.20,2023-12-31 16:32:00,18.10,80,8.80,1015,52,18.90,1.00,16,804,305,4.40,4.15,early
11778,UA,ORD,114,-10,2023-12-31 21:09:00,-4.30,100,-3.90,1000,88,-2.20,0.00,16,804,360,3.30,1.50,2023-12-31 19:15:00,-5.40,87,-2.30,993,82,0.40,0.00,13,804,315,10.70,7.03,early


In [38]:
from sklearn.preprocessing import OneHotEncoder

def get_ohe(df, col):
    ohe = OneHotEncoder(drop='first', handle_unknown='error', sparse_output=False, dtype='int')
    ohe.fit(df[[col]])
    temp_df = pd.DataFrame(data=ohe.transform(df[[col]]), columns=ohe.get_feature_names_out())
    df.drop(columns=[col], axis=1, inplace=True)
    df = pd.concat([df.reset_index(drop=True), temp_df], axis=1)
    return df

def add_custom_features(model_df):
    model_df['scheduled_departure_hour'] = model_df['scheduled_departure_time'].dt.hour
    model_df['scheduled_departure_day'] = model_df['scheduled_departure_time'].dt.day
    model_df['scheduled_departure_month'] = model_df['scheduled_departure_time'].dt.month
    model_df['scheduled_departure_weekday'] = model_df['scheduled_departure_time'].dt.weekday

    model_df['scheduled_arrival_hour'] = model_df['scheduled_arrival_time'].dt.hour
    model_df['scheduled_arrival_day'] = model_df['scheduled_arrival_time'].dt.day
    model_df['scheduled_arrival_month'] = model_df['scheduled_arrival_time'].dt.month
    model_df['scheduled_arrival_weekday'] = model_df['scheduled_arrival_time'].dt.weekday

    weather_features = ['app_temp', 'clouds', 'dewpt', 'pres', 'rh', 'temp', 'uv', 'vis','weather.code', 'wind_dir', 'wind_gust_spd', 'wind_spd']
    
    for feature in weather_features:
        model_df[f'{feature}_diff'] = model_df[f'{feature}_x'] - model_df[f'{feature}_y']

    # carrier_dummies = pd.get_dummies(model_df['Carrier Code'], prefix='carrier')
    # airport_dummies = pd.get_dummies(model_df['Origin Airport'], prefix='airport')


    # model_df = pd.concat([model_df, carrier_dummies, airport_dummies], axis=1)

    model_df['dep_time_day_prop'] = model_df['scheduled_departure_time'].dt.hour * 60 + model_df['scheduled_departure_time'].dt.minute
    model_df['dep_time_day_prop'] /= 1440  

    model_df['arr_time_day_prop'] = model_df['scheduled_arrival_time'].dt.hour * 60 + model_df['scheduled_arrival_time'].dt.minute
    model_df['arr_time_day_prop'] /= 1440  


    model_df['night_flight'] = model_df['scheduled_departure_time'].dt.hour.apply(lambda x: 1 if x >= 20 or x < 6 else 0)

    model_df['extreme_weather_temp_low'] = model_df['temp_x'].apply(lambda x: 1 if x < -10 else 0)
    model_df['extreme_weather_temp_high'] = model_df['temp_x'].apply(lambda x: 1 if x > 35 else 0)
    model_df['extreme_weather_wind'] = model_df['wind_gust_spd_x'].apply(lambda x: 1 if x > 50 else 0)
    
    model_df['time_of_day'] = model_df['scheduled_arrival_hour'].apply(categorize_hour)

    model_df["peak_zone"] = "peak"

    model_df.loc[((model_df['time_of_day']=="Morning") | (model_df['time_of_day']=="Early Morning") | (model_df['time_of_day']=="Afternoon")), "peak_zone"] = "off_peak"
    
    model_df["origin_airport"] = model_df["Origin Airport"]

    categorical_cols = ["Origin Airport", 'peak_zone']
    for i in categorical_cols:
        model_df = get_ohe(model_df, i)

    # model_df = pd.get_dummies(model_df, columns=categorical_cols)
    bool_cols = model_df.select_dtypes(include=["bool"]).columns
    for i in bool_cols:
        model_df[i] *= 1
    
    return model_df

def categorize_hour(hour):
    if 0 <= hour <= 3:
        return 'Early Morning'
    elif 4 <= hour <= 7:
        return 'Morning'
    elif 8 <= hour <= 11:
        return 'Late Morning'
    elif 12 <= hour <= 15:
        return 'Afternoon'
    elif 16 <= hour <= 19:
        return 'Evening'
    elif 20 <= hour <= 23:
        return 'Night'





In [39]:
model_df = add_custom_features(model_df)
numeric_cols = model_df.select_dtypes(include=['number']).columns.tolist()
drop_cols = model_df.select_dtypes(exclude=['number']).columns.tolist()
drop_cols.append('Arrival Delay (Minutes)')

In [40]:
numeric_cols.remove('Arrival Delay (Minutes)')

X = model_df.drop(columns=drop_cols, axis=1)
y = model_df['arrival_status']

scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])


In [41]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [42]:
X_train.shape

(9424, 54)

In [43]:
temp_df = pd.read_csv("./data/modified/initial_predictions_m1.csv")
# temp_df["scheduled_departure_time"] = pd.to_datetime(temp_df["scheduled_departure_time"])
# temp_df["date"] = temp_df["scheduled_departure_time"].dt.date
# temp_df = temp_df.groupby(["date"]).apply(lambda x: x.sort_values('Origin Airport')).reset_index(drop=True)
# model1_df = temp_df.iloc[::2]
# model1_df
temp_df

,Origin Airport,scheduled_departure_time,scheduled_arrival_time,Carrier Code,app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,Scheduled Elapsed Time (Minutes)
0,MCO,2024-04-10 11:05:00,2024-04-10 13:45:00,WN,17.20,84,9.40,997.50,60,17.20,2.90,19.70,804,280,6.90,4.40,29.10,57,16.40,"1,013.50",47,28.80,6.10,35.39,803,150,9.70,6.40,160.00
1,JFK,2024-04-10 13:33:00,2024-04-10 14:50:00,B6,17.10,82,9.10,998.00,59,17.10,2.50,18.72,804,283,6.50,4.26,12.30,91,8.20,"1,015.00",76,12.30,2.20,24.13,804,93,4.23,3.60,77.00
2,ORD,2024-04-10 18:52:00,2024-04-10 21:47:00,UA,12.60,76,6.60,999.00,67,12.60,0.00,24.13,804,110,4.06,2.53,12.30,92,3.30,981.00,54,12.30,0.00,24.13,804,60,4.40,2.80,175.00
3,MCO,2024-04-11 11:35:00,2024-04-11 14:20:00,WN,18.00,92,11.10,988.50,64,18.00,2.60,24.13,804,140,16.40,10.80,32.70,57,19.80,"1,007.00",52,30.80,6.10,24.13,803,200,19.60,12.80,165.00
4,JFK,2024-04-11 13:33:00,2024-04-11 14:50:00,B6,18.10,89,10.90,987.50,63,18.10,2.20,21.47,804,143,17.20,11.33,13.20,95,9.80,"1,009.00",80,13.20,2.00,24.13,804,153,12.00,8.00,77.00
5,ORD,2024-04-11 18:52:00,2024-04-11 21:47:00,UA,17.10,89,11.50,982.00,70,17.10,0.00,2.86,301,150,16.60,11.06,9.20,82,5.20,972.50,76,9.20,0.00,7.67,804,310,16.50,10.80,175.00
6,MCO,2024-04-12 11:35:00,2024-04-12 14:20:00,WN,14.90,82,7.00,972.50,59,14.90,3.00,17.46,804,230,14.10,9.67,24.90,1,10.00,"1,016.50",38,25.30,9.90,24.13,800,300,9.30,6.07,165.00
7,JFK,2024-04-12 13:33:00,2024-04-12 14:50:00,B6,14.20,81,6.30,972.50,59,14.20,2.60,24.13,804,230,14.15,9.63,15.60,58,8.60,993.50,63,15.60,3.50,24.13,803,223,14.95,10.17,77.00
8,ORD,2024-04-12 18:52:00,2024-04-12 21:47:00,UA,9.50,78,3.10,975.00,64,9.50,0.00,13.57,804,236,14.10,9.40,10.30,12,0.90,990.00,52,10.30,0.00,24.13,801,295,12.15,8.17,175.00
9,JFK,2024-04-13 13:12:00,2024-04-13 14:30:00,B6,8.50,76,1.60,988.00,62,8.50,3.40,15.12,804,280,15.40,10.27,12.90,61,2.20,"1,004.50",48,12.90,4.20,24.13,803,280,14.20,9.47,78.00


In [44]:
def read_actual_m1_files(period):
    df = pd.read_csv(f"./data/modified/{period}_predictions_m1.csv")
    df = convert_to_timestamp(df)
    df.loc[(df["Origin Airport"]=="JFK")&(df["Carrier Code"]=="DL"), "Carrier Code"] = "9E"
    df = add_custom_features(df)
    num_cols = df.select_dtypes(include=["number"]).columns.tolist()
    scaler = StandardScaler()
    df[num_cols] = scaler.fit_transform(df[num_cols])
    cat_df = df.select_dtypes(exclude=['number'])
    cat_df.loc[(df["origin_airport"]=="JFK")&(df["Carrier Code"]=="9E"), "Carrier Code"] = "DL"
    d_cols = cat_df.columns.tolist()
    df.drop(columns=d_cols, inplace=True)
    df = df[X_train.columns]
    
    return df, cat_df

In [45]:
initial_df, initial_cat_df = read_actual_m1_files("initial")
final_df, final_cat_df = read_actual_m1_files("final")

In [46]:
initial_df

,Scheduled Elapsed Time (Minutes),app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,scheduled_departure_hour,scheduled_departure_day,scheduled_departure_month,scheduled_departure_weekday,scheduled_arrival_hour,scheduled_arrival_day,scheduled_arrival_month,scheduled_arrival_weekday,app_temp_diff,clouds_diff,dewpt_diff,pres_diff,rh_diff,temp_diff,uv_diff,vis_diff,weather.code_diff,wind_dir_diff,wind_gust_spd_diff,wind_spd_diff,dep_time_day_prop,arr_time_day_prop,night_flight,extreme_weather_temp_low,extreme_weather_temp_high,extreme_weather_wind,Origin Airport_MCO,Origin Airport_ORD,peak_zone_peak
0,0.48,0.86,0.45,0.74,1.16,-0.82,0.90,0.87,0.14,0.30,0.96,-1.36,-1.26,1.51,0.06,1.54,0.88,-0.63,1.54,0.89,2.04,0.30,-0.41,-0.37,-0.30,-1.12,-1.34,0.00,-1.34,-1.03,-1.34,0.00,-1.34,-0.96,0.03,-1.27,-0.17,0.40,-1.01,-0.74,-2.27,0.30,0.86,-1.29,-1.42,-1.22,-0.99,0.00,0.00,0.00,0.00,1.41,-0.71,-0.85
1,-1.41,0.83,0.11,0.66,1.22,-1.08,0.88,0.56,-0.03,0.30,1.01,-1.45,-1.31,-0.69,1.11,0.04,0.97,1.43,-0.70,-0.37,0.08,1.03,-1.02,-1.59,-1.23,-0.41,-1.34,0.00,-1.34,-0.73,-1.34,0.00,-1.34,1.17,-1.15,0.53,-0.22,-1.74,1.16,0.85,-0.13,0.29,1.38,0.30,-0.13,-0.40,-0.67,0.00,0.00,0.00,0.00,-0.71,-0.71,-0.85
2,0.82,-0.13,-0.89,0.00,1.33,1.00,-0.21,-1.37,0.88,0.30,-1.74,-2.03,-1.88,-0.69,1.14,-0.86,-1.23,-0.13,-0.70,-1.08,0.08,1.03,-1.37,-1.56,-1.49,1.36,-1.34,0.00,-1.34,1.39,-1.34,0.00,-1.34,0.59,-1.38,1.07,1.79,0.40,0.56,0.71,0.99,0.29,0.16,-0.52,-0.58,1.36,1.39,0.00,0.00,0.00,0.00,-0.71,1.41,1.18
3,0.59,1.03,1.78,1.19,0.18,0.22,1.10,0.64,0.88,0.30,-1.27,0.87,0.85,1.99,0.06,2.16,0.45,-0.27,1.81,0.89,0.08,0.30,0.13,1.84,1.84,-1.12,-0.45,0.00,-0.45,-0.73,-0.45,0.00,-0.45,-1.32,0.29,-1.65,-0.31,0.33,-1.17,-0.88,0.99,0.30,-0.80,-1.41,-1.42,-1.05,-0.81,0.00,0.00,0.00,0.00,1.41,-0.71,-0.85
4,-1.41,1.05,1.28,1.14,0.07,-0.04,1.12,0.33,0.44,0.30,-1.22,1.06,1.03,-0.57,1.23,0.33,0.58,1.71,-0.57,-0.43,0.08,1.03,-0.37,0.14,0.24,-0.41,-0.45,0.00,-0.45,-0.73,-0.45,0.00,-0.45,1.18,-1.05,0.57,-0.48,-1.74,1.17,0.80,0.44,0.29,-0.36,1.21,1.16,-0.40,-0.67,0.00,0.00,0.00,0.00,-0.71,-0.71,-0.85
5,0.82,0.83,1.28,1.30,-0.53,1.78,0.88,-1.37,-2.70,-3.32,-1.11,0.92,0.94,-1.09,0.83,-0.51,-1.78,1.43,-1.12,-1.08,-2.79,1.03,1.31,1.15,1.17,1.36,-0.45,0.00,-0.45,1.39,-0.45,0.00,-0.45,1.56,-0.63,1.75,1.30,-0.95,1.57,0.71,-0.01,-3.32,-1.67,-0.38,-0.32,1.36,1.39,0.00,0.00,0.00,0.00,-0.71,1.41,1.18
6,0.59,0.36,0.11,0.11,-1.57,-1.08,0.34,0.94,-0.24,0.30,0.17,0.33,0.48,0.96,-1.68,0.37,1.07,-1.26,1.07,2.12,0.08,-1.87,1.20,-0.46,-0.41,-1.12,0.45,0.00,0.45,-0.73,0.45,0.00,0.45,-0.72,1.80,-0.36,-1.78,0.98,-0.85,-2.42,-0.39,0.32,-0.89,1.09,1.29,-1.05,-0.81,0.00,0.00,0.00,0.00,1.41,-0.71,-0.85
7,-1.41,0.22,-0.06,-0.07,-1.57,-1.08,0.18,0.64,0.88,0.30,0.17,0.35,0.47,-0.25,0.09,0.11,-0.42,0.51,-0.25,0.05,0.08,0.30,0.38,0.80,0.96,-0.41,0.45,0.00,0.45,-0.73,0.45,0.00,0.45,0.38,-0.10,-0.20,-0.45,-0.81,0.34,0.30,0.99,0.30,-0.22,-0.66,-0.71,-0.40,-0.67,0.00,0.00,0.00,0.00,-0.71,-0.71,-0.85
8,0.82,-0.79,-0.56,-0.92,-1.30,0.22,-0.96,-1.37,-0.89,0.30,0.26,0.33,0.39,-0.95,-1.34,-1.30,-0.65,-0.27,-0.97,-1.08,0.08,-1.15,1.15,0.18,0.29,1.36,0.45,0.00,0.45,1.39,0.45,0.00,0.45,0.45,1.31,0.82,-0.11,0.33,0.42,0.71,-1.20,0.31,-0.79,0.20,0.14,1.36,1.39,0.00,0.00,0.00,0.00,-0.71,1.41,1.18
9,-1.39,-1.00,-0.89,-1.31,0.13,-0.30,-1.20,1.25,-0.63,0.30,0.96,0.64,0.68,-0.61,0.18,-1.06,0.29,-0.55,-0.62,0.28,0.08,0.30,0.99,0.63,0.73,-0.41,1.34,0.00,1.34,-0.73,1.34,0.00,1.34,-0.01,-0.37,0.19,-0.19,0.48,-0.06,0.35,-0.88,0.30,-0.28,-0.04,-0.06,-0.52,-0.76,0.00,0.00,0.00,0.00,-0.71,-0.71,-0.85


In [47]:
initial_cat_df

,scheduled_departure_time,scheduled_arrival_time,Carrier Code,time_of_day,origin_airport
0,2024-04-10 11:05:00,2024-04-10 13:45:00,WN,Afternoon,MCO
1,2024-04-10 13:33:00,2024-04-10 14:50:00,B6,Afternoon,JFK
2,2024-04-10 18:52:00,2024-04-10 21:47:00,UA,Night,ORD
3,2024-04-11 11:35:00,2024-04-11 14:20:00,WN,Afternoon,MCO
4,2024-04-11 13:33:00,2024-04-11 14:50:00,B6,Afternoon,JFK
5,2024-04-11 18:52:00,2024-04-11 21:47:00,UA,Night,ORD
6,2024-04-12 11:35:00,2024-04-12 14:20:00,WN,Afternoon,MCO
7,2024-04-12 13:33:00,2024-04-12 14:50:00,B6,Afternoon,JFK
8,2024-04-12 18:52:00,2024-04-12 21:47:00,UA,Night,ORD
9,2024-04-13 13:12:00,2024-04-13 14:30:00,B6,Afternoon,JFK


In [48]:
set(X_train.columns.tolist())-set(initial_df.columns.tolist())

set()

In [49]:
set(initial_df.columns.tolist())-set(final_df.columns.tolist())

set()

In [50]:
print(X_train.columns)
print(initial_df.columns)
print(final_df.columns)

Index(['Scheduled Elapsed Time (Minutes)', 'app_temp_x', 'clouds_x', 'dewpt_x',
       'pres_x', 'rh_x', 'temp_x', 'uv_x', 'vis_x', 'weather.code_x',
       'wind_dir_x', 'wind_gust_spd_x', 'wind_spd_x', 'app_temp_y', 'clouds_y',
       'dewpt_y', 'pres_y', 'rh_y', 'temp_y', 'uv_y', 'vis_y',
       'weather.code_y', 'wind_dir_y', 'wind_gust_spd_y', 'wind_spd_y',
       'scheduled_departure_hour', 'scheduled_departure_day',
       'scheduled_departure_month', 'scheduled_departure_weekday',
       'scheduled_arrival_hour', 'scheduled_arrival_day',
       'scheduled_arrival_month', 'scheduled_arrival_weekday', 'app_temp_diff',
       'clouds_diff', 'dewpt_diff', 'pres_diff', 'rh_diff', 'temp_diff',
       'uv_diff', 'vis_diff', 'weather.code_diff', 'wind_dir_diff',
       'wind_gust_spd_diff', 'wind_spd_diff', 'dep_time_day_prop',
       'arr_time_day_prop', 'night_flight', 'extreme_weather_temp_low',
       'extreme_weather_temp_high', 'extreme_weather_wind',
       'Origin Airport_MCO',

In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd


models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42,n_estimators = 250,max_depth = 50, learning_rate= 1 ),
    'ada' : AdaBoostClassifier(random_state=42),
    'knn' : KNeighborsClassifier(n_neighbors=15),
    'dtree':  DecisionTreeClassifier(max_depth=7,random_state=42),
    'GNB':  GaussianNB(),
    'rf_tuned': RandomForestClassifier(random_state=42, n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_depth=20, criterion='gini', bootstrap=True)
}



In [80]:
for model_name, model in models.items():
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f'{model_name} Accuracy: {accuracy:.2f}')
        
    except Exception as e:
        print(e)

/Users/saikeerthanpalavarapu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

Logistic Regression Accuracy: 0.56


DecisionTreeClassifier(random_state=42)

Decision Tree Accuracy: 0.46


...

dtree Accuracy: 0.56


GaussianNB()

GNB Accuracy: 0.28


RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=200,
                       random_state=42)

rf_tuned Accuracy: 0.57


### Here, Tuned Random forest classifier has better accuracy among other models. So, Considering tuned Random forest classifier for predictions

In [69]:
rf = models["rf_tuned"]

rf = rf.fit(X_train, y_train) 
rf.score(X_train, y_train) 

test_output = pd.DataFrame(rf.predict(X_test), index = X_test.index, columns = ['pred_Y'])

test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
print('Fraction of correct classification ')
rf.score(X_test, y_test) 


0.9405772495755518

,pred_Y,arrival_status
6686,early,early
4080,early,early
4782,late,early
8188,early,late
2847,early,late


Fraction of correct classification 


0.5713073005093379

In [75]:
initial_pred_df = pd.DataFrame(rf.predict(initial_df), index = initial_df.index, columns = [f'initial_pred'])
final_pred_df = pd.DataFrame(rf.predict(final_df), index = initial_df.index, columns = [f'final_pred'])
        

In [76]:
initial_predictions = pd.concat(merge_in_cols, axis=1)
final_predictions = pd.concat(merge_fin_cols, axis=1)


In [77]:
pd.concat([initial_cat_df, initial_pred_df], axis=1)

,scheduled_departure_time,scheduled_arrival_time,Carrier Code,time_of_day,origin_airport,initial_pred
0,2024-04-10 11:05:00,2024-04-10 13:45:00,WN,Afternoon,MCO,early
1,2024-04-10 13:33:00,2024-04-10 14:50:00,B6,Afternoon,JFK,early
2,2024-04-10 18:52:00,2024-04-10 21:47:00,UA,Night,ORD,early
3,2024-04-11 11:35:00,2024-04-11 14:20:00,WN,Afternoon,MCO,late
4,2024-04-11 13:33:00,2024-04-11 14:50:00,B6,Afternoon,JFK,early
5,2024-04-11 18:52:00,2024-04-11 21:47:00,UA,Night,ORD,late
6,2024-04-12 11:35:00,2024-04-12 14:20:00,WN,Afternoon,MCO,early
7,2024-04-12 13:33:00,2024-04-12 14:50:00,B6,Afternoon,JFK,early
8,2024-04-12 18:52:00,2024-04-12 21:47:00,UA,Night,ORD,early
9,2024-04-13 13:12:00,2024-04-13 14:30:00,B6,Afternoon,JFK,early


In [78]:
pd.concat([final_cat_df, final_pred_df], axis=1)

,scheduled_departure_time,scheduled_arrival_time,Carrier Code,time_of_day,origin_airport,final_pred
0,2024-04-19 18:52:00,2024-04-19 21:47:00,UA,Night,ORD,early
1,2024-04-20 18:52:00,2024-04-20 21:47:00,UA,Night,ORD,early
2,2024-04-21 19:59:00,2024-04-21 22:52:00,AA,Night,ORD,early
3,2024-04-22 19:59:00,2024-04-22 22:52:00,AA,Night,ORD,early
4,2024-04-19 13:34:00,2024-04-19 14:51:00,B6,Afternoon,JFK,early
5,2024-04-20 13:25:00,2024-04-20 14:41:00,B6,Afternoon,JFK,early
6,2024-04-21 14:55:00,2024-04-21 16:21:00,DL,Evening,JFK,early
7,2024-04-22 14:55:00,2024-04-22 16:21:00,DL,Evening,JFK,early
8,2024-04-19 11:35:00,2024-04-19 14:20:00,WN,Afternoon,MCO,early
9,2024-04-20 13:35:00,2024-04-20 16:25:00,B6,Evening,MCO,early
